In [2]:
# ====================================================================================
# This notebook provides a complete, runnable example of building a simple
# large language model (LLM) from scratch using PyTorch.
# It is designed to be run in a Google Colab environment.
#
# Before you begin:
# 1. Go to the 'Runtime' menu in Colab.
# 2. Select 'Change runtime type'.
# 3. Choose 'T4 GPU' or 'V100 GPU' as the hardware accelerator.
# ====================================================================================

import torch
import torch.nn as nn
from torch.nn import functional as F

# --- Step 1: Data Preparation and Tokenization ---
# This part handles loading a text file and converting characters into numbers (tokenization).

# Download a sample text file (The Adventures of Sherlock Holmes)
# You can replace this with any large text file.
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O sherlock.txt
with open('sherlock.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Create a vocabulary of all unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)

# Create mappings from character to integer and back
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
def encode(s):
    """Encodes a string into a list of integers."""
    return [stoi[c] for c in s]
def decode(l):
    """Decodes a list of integers into a string."""
    return "".join([itos[i] for i in l])

# --- Step 2: Hyperparameters and Data Loading ---
# These are the main settings for our model and training process.

# Hyperparameters
batch_size = 64        # Number of sequences to process in parallel
block_size = 256       # Maximum context length for predictions
max_iters = 5000       # Number of training steps
eval_interval = 500    # How often to evaluate loss
learning_rate = 3e-4   # Model learning rate
eval_iters = 200       # Number of batches to use for evaluation
n_embd = 384           # Embedding dimension
n_head = 6             # Number of attention heads
n_layer = 6            # Number of transformer blocks
dropout = 0.2          # Dropout rate

# Set device to GPU if available, otherwise CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Convert text to a PyTorch tensor
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# Data loading function
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# --- Step 3: Model Architecture (GPT-style Transformer) ---
# This defines the building blocks of our LLM.

class Head(nn.Module):
    """ One self-attention head """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """ Multiple heads of self-attention in parallel """
    def __init__(self, n_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ A simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class TransformerBlock(nn.Module):
    """ A single Transformer block: communication followed by computation """
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class SimpleGPT(nn.Module):
    """ The main GPT model class """
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[TransformerBlock(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# --- Step 4: Training Loop ---
# This section initializes the model and runs the training process.

model = SimpleGPT()
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

print(f"Training on device: {device}")

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# --- Step 5: Text Generation ---
# After training, use the model to generate new text.

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = decode(model.generate(context, max_new_tokens=200)[0].tolist())
print("\nGenerated Text:")
print("----------------")
print(generated_text)

--2025-08-09 16:34:38--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘sherlock.txt’

sherlock.txt        100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2025-08-09 16:34:38 (45.2 MB/s) - ‘sherlock.txt’ saved [1115394/1115394]

Training on device: cuda
Step 0: train loss 4.3039, val loss 4.3069
Step 500: train loss 2.0374, val loss 2.1071
Step 1000: train loss 1.6273, val loss 1.7954
Step 1500: train loss 1.4564, val loss 1.6573
Step 2000: train loss 1.3588, val loss 1.5735
Step 2500: train loss 1.2951, val loss 1.5357
Step 3000: train loss 1.2433, val loss 1.5143
Step 3500: train loss 1.2005, val loss 1.4955
Step 4